In [2]:
import itertools

#list of links

urls = ["https://www.derstandard.at/search?query=verfassungsgerichtshof&fd=2000-01-01&td=2004-12-31&s=date&p=", "https://www.derstandard.at/search?query=verfassungsgerichtshof&fd=2005-01-01&td=2008-12-31&s=date&p=", "https://www.derstandard.at/search?query=verfassungsgerichtshof&fd=2009-01-01&td=2012-12-31&s=date&p=", "https://www.derstandard.at/search?query=verfassungsgerichtshof&fd=2013-01-01&td=2016-12-31&s=date&p=", "https://www.derstandard.at/search?query=verfassungsgerichtshof&fd=2017-01-01&td=2021-12-31&s=date&p="]

# need to combine urls_p and pagenumber
urls_complete = []

for url in urls:
    urls_complete.append([url+ str(i) for i in range(1,17+1)])
    
del(urls_complete[2][16]) # two time frames have one page less
del(urls_complete[3][16])
    
urls_complete = list(itertools.chain(*urls_complete))

In [3]:
len(urls_complete)

83

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [34]:
#set driver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [35]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

#open standard homepage
driver.get("https://www.derstandard.at")

#accept cookies
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='SP Consent Message']")))
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//button[@title='Einverstanden']"))).click()

#deny push notifications
driver.switch_to.default_content()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Nein, danke']"))).click()

True

In [ ]:
# works well, but i wonder whether this could be done faster with swifter or multiprocess
from selenium.webdriver.common.by import By

links = []
for url in urls_complete:
    driver.get(url)
    
    # link 
    a = driver.find_elements(By.XPATH, "//div[@class='teaser-inner']//child::a")    
    links.append([lnk.get_attribute("href") for lnk in a])

links = list(itertools.chain(*links))

print(links)

In [5]:
import pyarrow.feather as feather
import pandas as pd

df_links = pd.DataFrame(links)

feather.write_feather(df_links, 'links.feather')

NameError: name 'links' is not defined

In [27]:
import pyarrow.feather as feather
import pandas as pd
import random
from selenium.webdriver.common.by import By

df = pd.read_feather('links.feather')
df.rename(columns={0: "links"})

links = df[0].values.tolist()

,links
0,https://www.derstandard.at/story/1892286/harth...
1,https://www.derstandard.at/story/1898042/gruen...
2,https://www.derstandard.at/story/1869686/wunsc...
3,https://www.derstandard.at/story/1887556/aufge...
4,https://www.derstandard.at/story/1898112/buerg...
...,...
7790,https://www.derstandard.at/story/2000050299493...
7791,https://www.derstandard.at/story/2000050322996...
7792,https://www.derstandard.at/story/2000050315153...
7793,https://www.derstandard.at/story/2000049996368...


In [30]:
live = r'jetzt/livebericht'

links_short = links
len(links_short)
len(links)

7795

7795

In [31]:
livebericht = []

for elem in links_short:
        if live in elem:
            livebericht.append(elem)
        if live in elem:
            links_short.remove(elem)
            
len(links_short)
print(livebericht)

7770

['https://www.derstandard.at/jetzt/livebericht/2000122736430/doch-noch-termine-fuer-massentests-in-wien-vor-weihnachten-regierung', 'https://www.derstandard.at/jetzt/livebericht/2000115867752/coronavirus-auch-soelden-und-st-christoph-unter-quarantaene-eu-beschliesst']


In [ ]:
# now working with soup get html of all pages
import requests
import random
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}

linkstest = random.sample(links, 50)
soups = []
for link in linkstest:
    page = requests.get(link, headers = headers)
    soups.append(BeautifulSoup(page.content, "html.parser"))

In [ ]:
len(soups)
len(linkstest)

In [ ]:
# trying this with soup
from datetime import datetime
import pandas as pd
import numpy as np

start_time = datetime.now()

article = []
article2 = []
title = []
author = []
commentno = []
time = []

for soup in soups:
    article1 = soup.find('div', class_="article-body").findAll('p')
    #article.append(article1.text if article1 else "N/A")
    #article3 = article1.find('p')
    #article2.append(article3.text if article3 else "N/A")

    
    title1 = soup.find('h1', class_="article-title")
    title.append(title1.text if title1 else "N/A")
    
    author1 = soup.find('div', class_="article-origins")
    author.append(author1.text if author1 else "N/A")
    
    print(article1)
        
#df_articles=pd.DataFrame({'article': article,
 #                         'article2': article2,
  #                        'title': title,
   #                       'author': author})
    

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [34]:
from selenium.common.exceptions import NoSuchElementException

linkstest = random.sample(links, 2)

article = []
title = []
author = []

for link in linkstest:
    driver.get(link)
    
    try:
        articles = driver.find_elements(By.CLASS_NAME, 'article-body')
        authors = driver.find_elements(By.CLASS_NAME, 'article-origin')
        titles = driver.find_elements(By.CLASS_NAME, 'article-title')
    except NoSuchElementException:
        articles = None
        titles = None
        authors = None
        
    if authors is not None:
        for value in authors:
            row["author"] = value.text
    else:
        row["author"] = "N/A"

  #  for value in titles:
   #     title.append(value.text)    
    #for value in authors:
     #   author.append(value.text if value.text != "" else "N/A")

#df_articles=pd.DataFrame({'article': article,
 #                        'title': title,
  #                       'author': author})
    

print(author)

[]


In [ ]:
linkstest = random.sample(links, 2)

article = []
title = []
author = []

for link in linkstest:
    driver.get(link)
    
    articles = driver.find_elements(By.CLASS_NAME, 'article-body')
    for value in articles:
        article.append(value.text)
    titles = driver.find_elements(By.CLASS_NAME, 'article-title')
    for value in titles:
        title.append(value.text)
    authors = driver.find_elements(By.CLASS_NAME, 'article-origin')
    try:
        authors = driver.find_elements(By.CLASS_NAME, 'article-origin')
        except (NoSuchElementException, StaleElementReferenceException):
        authors = None
    for value in authors:
        if authors is not None:
          author.append(value.text)
        else:
          author.append("N/A")
        author.append(value.text if value.text else "N/A")
        print(value.text)


    try:
        brand_name = driver.find_element_by_xpath('//*[@class="a-spacing-small"][.//*[contains(.,"Brand")]]/td[@class="a-span9"]/span')
    except (NoSuchElementException, StaleElementReferenceException):
        brand_name = None

In [ ]:
links = []
for url in urls_complete:
    driver.get(url)
    
    # link 
    a = driver.find_elements(By.XPATH, "//div[@class='teaser-inner']//child::a")    
    links.append([lnk.get_attribute("href") for lnk in a])

In [ ]:
len(article)
len(title)
len(author)
len(soups)

df_articles['links'] = linkstest
print(df_articles)

In [ ]:
from lxml import html

page = requests.get(link)
tree = html.fromstring(page.content) 
buyers = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "article-body", " " ))]')
print(buyers)

In [ ]:
print(buyers)

In [ ]:
from IPython.display import HTML

HTML(df_articles.to_html(index=False))

In [ ]:
# json for date and comments
import re
import json

test = links[1]

soups_test = []
article = []

print(links)

for link in links:
    page = requests.get(link, headers = headers)
    soups_test.append(BeautifulSoup(page.content, "html.parser"))

for soup in soups_test:
    article1 = soup.find('div', class_="article-pubdate")
    article.append(article1.text if article1 else "N/A")
    


In [56]:
links1 = links_short[7000:-1]
len(links1)

769

In [57]:
# the rest with selenium
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import pandas as pd
import numpy as np

links1 = links_short[7001:-1]

df = pd.DataFrame(links1, columns = ['links'])

start_time = datetime.now()

def get_additions(row):        
    driver.get(row['links'])
    
    try:
        commentno = driver.find_element(By.CLASS_NAME, "article-postingcount").text
    except NoSuchElementException:
        commentno = "NA"
    try:
        time = driver.find_element(By.XPATH, "//time").text
    except NoSuchElementException:
        time = "NA"
    try:
        articles = driver.find_elements(By.CLASS_NAME, 'article-body')
        for value in articles:
            article = value.text
    except NoSuchElementException:
        article = "NA"
    try:
        titles = driver.find_elements(By.CLASS_NAME, 'article-title')
        for value in titles:
            title = value.text
    except NoSuchElementException:
        title = "NA"

    return pd.Series({
        'article': article,
        'title': title,
        'commentno': commentno,
        'time': time
    })

new_df6 = df.apply(get_additions, axis=1).join(df)
new_df6.head()

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

,article,title,commentno,time,links
0,Wien – Die Republik Österreich verlangt Schade...,Beamte müssen Staat für Pfuscherei bei Hofburg...,", \n246 Postings","27. März 2019, 14:46",https://www.derstandard.at/story/2000100318560...
1,Klagenfurt – Das Land Kärnten hat am Dienstag ...,Land will 500.000 Euro von Kärntner Grünen zurück,", \n41 Postings","26. März 2019, 13:55",https://www.derstandard.at/story/2000100246355...
2,Wien – Der Verfassungsgerichtshof (VfGH) hat e...,Pflegeregressentfall gilt auch für untergebrac...,", \n8 Postings","26. März 2019, 11:35",https://www.derstandard.at/story/2000100231502...
3,BILD NICHT MEHR VERFÜGBAR.\nFoto: Reuters\nDer...,Innenminister Kickl trommelt wieder für den Bu...,", \n59 Postings","24. März 2019, 15:54",https://www.derstandard.at/story/2000100120857...
4,In dieser Galerie: 2 Bilder\nWien – Das im Isl...,VfGH weist Beschwerden ausgewiesener Imame ab,", \n355 Postings","21. März 2019, 11:41",https://www.derstandard.at/story/2000099944613...


Duration: 0:20:38.079982


In [29]:
feather.write_feather(new_df, 'articles1.feather')
feather.write_feather(new_df2, 'articles2.feather')
feather.write_feather(new_df3, 'articles3.feather')

In [38]:
new_df4 = new_df_short
feather.write_feather(new_df4, 'articles4.feather')

In [54]:
feather.write_feather(new_df5, 'articles5.feather')

In [58]:
feather.write_feather(new_df6, 'articles6.feather')

In [21]:
from IPython.display import HTML

HTML(new_df.to_html(index=False))

In [ ]:
import pyarrow.feather as feather

feather.write_feather(new_df, 'dates.feather')

In [ ]:
import pyarrow.feather as feather

new_df.tofeather("dates.feather")
df_articles.tofeather("articles.feather")

### 